In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/dev/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    !pip cache purge
    %run colab_setup.py --branch "dev"

except ImportError:
    pass

## Convert a fNIRS dataset to BIDS

### Do not run all cells at once. Carefully read the comments before each code cell — some steps require you to manually modify certain files (e.g., the mapping CSV) before proceeding. Make sure all required edits are completed before continuing to the next step.

In [ ]:
import os
import pandas as pd
import re
import snirf2bids as s2b
from pathlib import Path
from cedalion.io import bids

### Provide file paths

This notebook shows how to convert an fNIRS dataset into BIDS format. To use it, provide the following inputs:

1.  Dataset Path: Folder containing the raw dataset.
2.  Destination Path: Folder where the BIDS-compliant dataset will be saved.
3.  Mapping CSV File: CSV file that defines the dataset structure and provides necessary details for BIDS conversion.
4.  (Optional) extra_meta_data File: Additional metadata to include in the description.json file. You can use [this google form](https://docs.google.com/forms/d/e/1FAIpQLSeZjlgIqCwp054HsHmTBKPziqcOlfTcaWpdXcGFYPDf0Q5vNg/viewform?usp=sf_link) or [this website](https://neurojson.org/Create/dataset_description_fnirs) to create this file.
5.  (Optional) participants.tsv / participants.json files. If you already have a participants.tsv/.json file and provide the link below, it will be used directly.
Alternatively, if you have participant-level metadata saved in a CSV or Excel file, with the first column as the participant ID and the remaining columns as metadata (with appropriate headers) and you provide the link to it below, the script will convert it into properly formatted .tsv and .json files for BIDS.

In [ ]:
dataset_path = 'path-to-your-dataset-folder'  # REQUIRED
destination_path = 'path-to-your-destination-bids-folder' # REQUIRED

dataset_path = Path(dataset_path)
destination_path = Path(destination_path)

In [ ]:
extra_meta_data_path = Path('path-to-your-meta-data')  # OPTIONAL
extra_meta_data_path = extra_meta_data_path if os.path.exists(extra_meta_data_path) else None

mapping_df_path = bids.get_snirf2bids_mapping_csv(dataset_path) 

In [ ]:
participants_tsv_file = 'path-to-your-participants.tsv' # OPTIONAL
participants_json_file = 'path-to-your-participants.json' # OPTIONAL

participants_tsv_file = Path(participants_tsv_file)
participants_json_file = Path(participants_json_file)

### Please modify the mapping CSV file which is automatically created under you raw dataset folder.

By default, a mapping CSV file is generated under the main raw dataset folder using the get_snirf2bids_mapping_csv function.
Before running the rest of the code, open this file, make any necessary edits, and save it. A valid mapping CSV must include all SNIRF files in your dataset, along with the following columns:

- sub: Participant identifier
- ses (optional): Session identifier
- task: Task name or label
- run (optional): Run number
- acq (optional): Acquisition label
- cond (optional): List of condition labels
- cond_match (optional): List of matching condition values
- duration (optional): Event duration in seconds


In [ ]:
mapping_df = pd.read_csv(mapping_df_path, dtype=str)
mapping_df.head(10)

The mapping table created below serves as a key component for organizing and processing your dataset. The `ses`, `run`, and `acq` columns are optional and can be set to None if not applicable. The `current_name` column contains the path to the SNIRF files in your dataset.

### Looking for possible *_scan.tsv files

To ensure no important information (e.g., acquisition time) from the original dataset is lost, we will:

- Search Subdirectories: Traverse through all subdirectories within the dataset.
- Locate Existing Scan Files: Search for all *_scan.tsv files in the dataset.
- Integrate into Mapping Table: Extract the relevant information from these files and add it to our mapping table.
- Extracts acquisition time from SNIRF files if missing in the `_scans.tsv` file.

This approach ensures that any details, such as acquisition time, are retained and incorporated into the BIDS-compliant structure.

In [ ]:
mapping_df["filename_org"] = mapping_df["current_name"].apply(
    lambda x: os.path.basename(x))
scan_df = bids.search_for_acq_time_in_scan_files(dataset_path)

mapping_df = pd.merge(mapping_df, scan_df, on="filename_org", how="left")
mapping_df["acq_time"] = mapping_df.apply(bids.search_for_acq_time_in_snirf_files, axis=1, args=(dataset_path,))

mapping_df.head(10)

The `acq_time` information is retrieved from the original dataset's *_scan.tsv files and integrated into the mapping table.

### Looking for possible *_session.tsv files

Similar to *_scan.tsv files, we search for *_session.tsv files in the dataset path to capture additional session-level metadata, such as acquisition times. Any relevant information from these files is added to the mapping table to ensure all session details are preserved.

In [ ]:
session_df = bids.search_for_sessions_acq_time(dataset_path)
mapping_df = pd.merge(mapping_df, session_df, on=["sub", "ses"], how="left")

mapping_df.head(10)

### Create BIDS Folder Structure

The goal of this section is to rename the SNIRF files according to the BIDS naming convention and place them in the appropriate directory under `destination_path`, following the BIDS folder structure.

Steps:
1. Generate new filenames: Create BIDS-compliant filenames for all SNIRF records.
2. Determine file locations: Identify the appropriate locations for these files within the BIDS folder hierarchy.

This process ensures that the dataset adheres to BIDS standards for organization and naming.

In [ ]:
mapping_df[["bids_name", "parent_path"]] = mapping_df.apply(
    bids.create_bids_standard_filenames, axis=1, result_type='expand')

mapping_df.head(10)

To facilitate proper organization:

- `parent_path`: Added to the mapping dataframe to define the location of each SNIRF file within `destination_path`.
- `bids_name`: Specifies the new BIDS-compliant name for each file.
In the following sections, we will rename all files to their corresponding `bids_name` and copy them to their designated parent_path.

In [ ]:
_ = mapping_df.apply(bids.copy_rename_snirf, axis=1, args=(dataset_path, destination_path))

### Create BIDS specific files (e.g., _coordsystem.json)

In this step, we utilize the snirf2bids Python package to generate the necessary .tsv and .json files for the BIDS structure.

For every record, the following files will be created:
1. _coordsystem.json
2. _optodes.json
3. _optodes.tsv
4. *_channels.tsv
5. *_events.json
6. *_events.tsv
7. *_nirs.json

These files are essential for ensuring the dataset adheres to BIDS standards.

In [ ]:
s2b.snirf2bids_recurse(destination_path)
pattern = re.compile(r'.*_scans\.tsv$|^participants\.tsv$|^temp_participants\.tsv$')
files_to_delete = [file for file in destination_path.rglob('*') if file.is_file() and pattern.match(file.name)]
for file in files_to_delete:
    file.unlink()

### Create _scan.tsv Files

Now, we proceed to create scan files for all subjects and sessions. Previously, we searched the original dataset path for any provided scan information, which will now be incorporated into the BIDS structure.

In [ ]:
scan_df = mapping_df[["sub", "ses", "bids_name", "acq_time"]]
scan_df['ses'].fillna("Unknown", inplace=True)
scan_df = scan_df.groupby(["sub", "ses"])
scan_df.apply(lambda group: bids.create_scan_files(group, destination_path))

### Create _session.tsv Files

The next step is to create session files for all subjects. As with the scan files, we previously searched the original dataset path for any session information, which will now be used to create the corresponding BIDS session files.

In [ ]:
session_df = mapping_df[["sub", "ses", "ses_acq_time"]]
session_df = session_df.groupby(["sub"])
session_df.apply(lambda group: bids.create_session_files(group, destination_path))

### Create and Integrate participants.tsv and participants.json

In this step, we gather available participant information and incorporate it into the BIDS structure. 

If you want to use custom participant metadata, you should provide it at the beginning of the code, either as a participants.tsv file or as a CSV/Excel file.

- If you provide a participants.tsv file but not a corresponding participants.json, you should fill out the participants.json manually to include descriptions for each field to comply with BIDS standards.

- If you provide neither file, new participants.tsv and participants.json files will be automatically created with standard fields:

    - species
    - age
    - sex
    - handedness

You can also pass your favourite/custom fields instead of these defaults when creating new files (only applies if no valid TSV is provided).

In [ ]:
saved_participants = bids.create_participants_files(bids_dir=destination_path, 
                                                    participants_tsv_path= participants_tsv_file, 
                                                    participants_json_path=participants_json_file, 
                                                    mapping_df=mapping_df,
                                                    fields=["gender", "age"])

### Create data description file

To create the dataset_description.json file, we follow these steps:

1. Search for an existing dataset_description.json in the dataset path and retain the provided information.
2. If extra_meta_data_path is specified, add the additional metadata about the dataset.
3. If neither dataset_description.json nor extra metadata is provided, use the basename of the dataset directory as the dataset name and set the BIDS version to '1.10.0'.

In [ ]:
bids.create_data_description(dataset_path, destination_path, extra_meta_data_path)

### Check _coordsystem.json file

Since an empty string is not allowed for the `NIRSCoordinateSystem` key in the *_coordsystem.json file, we will populate it with "Other" to ensure BIDS compliance.

In [ ]:
bids.check_coord_files(destination_path)

### Fix *_events.tsv order

Sorting events files based on onset time

In [ ]:
_ = mapping_df.apply(bids.sort_events, axis=1, args=(destination_path,))

### Edit *_events.tsv

To allow editing of the `duration` or `trial_type` columns in the *_events.tsv files, the mapping CSV file must include the following extra columns:

1. `duration`: Specifies the new duration for each SNIRF file that needs editing.
2. cond and cond_match:

    - cond: A list of existing condition labels found in the SNIRF file (e.g., [1, 2]).

    - cond_match: A list of new labels you want to use in place of those conditions (e.g., ["con", "inc"]).
    
These two columns will be combined into a dictionary to update the trial_type column in the events file. This allows for relabeling of condition names in a BIDS-compliant way.

In [ ]:
_ = mapping_df.apply(bids.edit_events, axis=1, args=(destination_path,))

### Creating sourcedata Directory

Finally there is this possiblity to keep your original data under sourcedata directory at your `destination_path`.

In [ ]:
bids.save_source(dataset_path, destination_path)